In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style='whitegrid')

In [10]:
inv_trans = pd.read_csv('InventoryTransactions.csv')
inv_trans.head()

C:\Users\jrstark\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,source,line_no,lev_no,seq_no,item_no,loc,trx_tm,doc_type,year,month,day
0,B,0,0,0,08020,TRN,1900-01-01 17:40:57.707,H,2017,8,31
1,B,0,0,1,080206,01,1900-01-01 17:40:57.707,H,2017,8,31
2,B,0,0,2,080206,TRN,1900-01-01 17:40:57.707,H,2017,8,31
3,B,0,0,3,08026,01,1900-01-01 17:40:57.707,H,2017,8,31
4,B,0,0,4,08026,TRN,1900-01-01 17:40:57.707,H,2017,8,31


In [11]:
inv_in = inv_trans[inv_trans['source'] == 'R'] 
inv_in = inv_in[inv_in['loc'] == 1]
print(inv_in.info())
inv_in[['source','loc']].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95441 entries, 2952522 to 3076856
Data columns (total 11 columns):
source      95441 non-null object
line_no     95441 non-null int64
lev_no      95441 non-null int64
seq_no      95441 non-null int64
item_no     95441 non-null object
loc         95441 non-null object
trx_tm      95441 non-null object
doc_type    95441 non-null object
year        95441 non-null int64
month       95441 non-null int64
day         95441 non-null int64
dtypes: int64(6), object(5)
memory usage: 8.7+ MB
None


source    1
loc       1
dtype: int64

In [12]:
from datetime import datetime

inv_in['hour_in'] = pd.to_datetime(inv_in['trx_tm']).dt.hour
inv_in['date'] = inv_in.apply(lambda row: datetime(row['year'], row['month'], row['day']), axis=1)
inv_in['day_of_week'] = inv_in['date'].dt.dayofweek
inv_in = inv_in[inv_in['loc'] == 1] 
inv_in.head()

,source,line_no,lev_no,seq_no,item_no,loc,trx_tm,doc_type,year,month,day,hour_in,date,day_of_week
2952522,R,1,2,0,12261OA,1,1900-01-01 09:54:57.000,O,2015,10,15,9,2015-10-15,3
2952523,R,6,2,0,40420OA,1,1900-01-01 11:09:57.000,O,2015,10,9,11,2015-10-09,4
2952528,R,1,2,0,MTTPROD,1,1900-01-01 14:22:25.000,R,2015,7,1,14,2015-07-01,2
2952529,R,1,2,0,MTTPROD,1,1900-01-01 14:27:36.000,R,2015,7,1,14,2015-07-01,2
2952530,R,1,2,0,MTTPROD,1,1900-01-01 14:32:04.000,R,2015,7,1,14,2015-07-01,2


In [13]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from datetime import timedelta

cal = calendar()
holidays_today = cal.holidays(start=inv_in['date'].min(), end=inv_in['date'].max())
holidays_yesterday = holidays_today + timedelta(days = 1)
holidays_tomorrow = holidays_today - timedelta(days = 1)


inv_in['Holiday_Today'] = inv_in['date'].isin(holidays_today)
inv_in['Holiday_Today'] = inv_in['Holiday_Today'].astype(int)

inv_in['Holiday_Yesterday'] = inv_in['date'].isin(holidays_yesterday)
inv_in['Holiday_Yesterday'] = inv_in['Holiday_Yesterday'].astype(int)

inv_in['Holiday_Tomorrow'] = inv_in['date'].isin(holidays_tomorrow)
inv_in['Holiday_Tomorrow'] = inv_in['Holiday_Tomorrow'].astype(int)

inv_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95441 entries, 2952522 to 3076856
Data columns (total 17 columns):
source               95441 non-null object
line_no              95441 non-null int64
lev_no               95441 non-null int64
seq_no               95441 non-null int64
item_no              95441 non-null object
loc                  95441 non-null object
trx_tm               95441 non-null object
doc_type             95441 non-null object
year                 95441 non-null int64
month                95441 non-null int64
day                  95441 non-null int64
hour_in              95441 non-null int64
date                 95441 non-null datetime64[ns]
day_of_week          95441 non-null int64
Holiday_Today        95441 non-null int32
Holiday_Yesterday    95441 non-null int32
Holiday_Tomorrow     95441 non-null int32
dtypes: datetime64[ns](1), int32(3), int64(8), object(5)
memory usage: 12.0+ MB


In [14]:
inv_in.drop(columns=['lev_no','seq_no','trx_tm','loc','source','line_no','doc_type'], inplace=True)
inv_in.head()

,item_no,year,month,day,hour_in,date,day_of_week,Holiday_Today,Holiday_Yesterday,Holiday_Tomorrow
2952522,12261OA,2015,10,15,9,2015-10-15,3,0,0,0
2952523,40420OA,2015,10,9,11,2015-10-09,4,0,0,0
2952528,MTTPROD,2015,7,1,14,2015-07-01,2,0,0,0
2952529,MTTPROD,2015,7,1,14,2015-07-01,2,0,0,0
2952530,MTTPROD,2015,7,1,14,2015-07-01,2,0,0,0


In [15]:
inventory_final = inv_in.copy()
inventory_final.head()

,item_no,year,month,day,hour_in,date,day_of_week,Holiday_Today,Holiday_Yesterday,Holiday_Tomorrow
2952522,12261OA,2015,10,15,9,2015-10-15,3,0,0,0
2952523,40420OA,2015,10,9,11,2015-10-09,4,0,0,0
2952528,MTTPROD,2015,7,1,14,2015-07-01,2,0,0,0
2952529,MTTPROD,2015,7,1,14,2015-07-01,2,0,0,0
2952530,MTTPROD,2015,7,1,14,2015-07-01,2,0,0,0


In [16]:
inventory_final['item_no'].describe()

count                              95441
unique                              2054
top       MTTPROD                       
freq                               16725
Name: item_no, dtype: object

In [17]:
###CREATE AN ITEM_ID_MAP TO MAP BACK TO SH ITEM NUMBERS WHICH ARE NOT ALWAYS NUMERIC###

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
inventory_final['item_id'] = le.fit_transform(inventory_final['item_no'])

item_id_map = inventory_final[['item_no','item_id']].copy()

In [18]:
#amount of inventory that comes in during different periods

inv_by_month = inventory_final.groupby(['month']).agg({'item_no':'count'})
inv_by_dow = inventory_final.groupby(['day_of_week']).agg({'item_no':'count'})
inv_by_hour = inventory_final.groupby(['hour_in']).agg({'item_no':'count'})
inv_by_dow_hour = inventory_final.groupby(['day_of_week','hour_in']).agg({'item_no':'count'})

In [19]:
#times when type of items of inventory come in

item_master = pd.read_excel('DD_ItemMaster.xlsx')
item_master = item_master[item_master['loc'] == 1]
item_master = item_master[['item_no','item_desc_1','p_and_ic_cd', 'prod_cat','loc']]
item_master.dropna(subset = ['p_and_ic_cd'], axis=0, inplace=True)
item_master.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5055 entries, 0 to 6254
Data columns (total 5 columns):
item_no        5055 non-null object
item_desc_1    5055 non-null object
p_and_ic_cd    5055 non-null object
prod_cat       5055 non-null object
loc            5055 non-null int64
dtypes: int64(1), object(4)
memory usage: 237.0+ KB


In [20]:
#item_master['item_no'].value_counts().max() #123, 4, 30
#item_master[item_master['item_no'] == '30']

#keep only those with a single item_no...

item_counts = item_master.groupby(['item_no']).agg({'item_no':'count'})
item_counts.rename(columns={'item_no':'freq'}, inplace=True)
item_master = pd.merge(item_master, item_counts, on='item_no', how='inner')
item_master = item_master[item_master['freq'] == 1]
item_master.drop(columns = ['freq','loc'], inplace=True)
item_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5055 entries, 0 to 5054
Data columns (total 4 columns):
item_no        5055 non-null object
item_desc_1    5055 non-null object
p_and_ic_cd    5055 non-null object
prod_cat       5055 non-null object
dtypes: object(4)
memory usage: 197.5+ KB


In [21]:
item_info = pd.merge(inventory_final, item_master, on='item_no', how='inner')
item_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91529 entries, 0 to 91528
Data columns (total 14 columns):
item_no              91529 non-null object
year                 91529 non-null int64
month                91529 non-null int64
day                  91529 non-null int64
hour_in              91529 non-null int64
date                 91529 non-null datetime64[ns]
day_of_week          91529 non-null int64
Holiday_Today        91529 non-null int32
Holiday_Yesterday    91529 non-null int32
Holiday_Tomorrow     91529 non-null int32
item_id              91529 non-null int32
item_desc_1          91529 non-null object
p_and_ic_cd          91529 non-null object
prod_cat             91529 non-null object
dtypes: datetime64[ns](1), int32(4), int64(5), object(4)
memory usage: 9.1+ MB


In [25]:
item_info.head()
item_info.to_excel('item_final.xlsx')

In [ ]:
#groupby time,day,month,day of week, etc... and include the 7 categories, or by product categories.

In [ ]:
#machine learning just by date, holiday, and time to predict broad incoming items